In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az

In [2]:
def simulate_death_rate_data(n_cells=500, n_periods=100, true_lambda=0.001, excess_zeros_prob=0.80, random_seed=42):
    """
    Simulate death rate data for a given number of cells over multiple time periods.

    Parameters:
    n_cells (int): Number of grid cells to simulate.
    n_periods (int): Number of time periods to simulate.
    true_lambda (float): True mean death rate per person.
    excess_zeros_prob (float): Probability of excess zeros in the data.
    random_seed (int): Random seed for reproducibility.

    Returns:
    pd.DataFrame: A DataFrame containing simulated death rate data with columns:
                  'cell_id', 'time_period', 'population', 'deaths', 'death_rate'.
    """
    np.random.seed(random_seed)

    # Calculate total observations
    total_observations = n_cells * n_periods

    # Simulate population data for each cell
    population = np.random.randint(1000, 10000, size=n_cells)
    population = np.repeat(population, n_periods)  # Repeat population for each time period

    # Simulate death counts with excess zeros for each cell over time periods
    death_counts = np.random.negative_binomial(true_lambda * population, 0.5)
    excess_zeros = np.random.binomial(1, excess_zeros_prob, size=total_observations)
    death_counts = death_counts * (1 - excess_zeros)

    # Create a DataFrame
    data = pd.DataFrame({
        'cell_id': np.tile(np.arange(n_cells), n_periods),
        'time_id': np.repeat(np.arange(n_periods), n_cells),
        'population': population,
        'deaths': death_counts
    })

    # Calculate death rates
    data['death_rate'] = data['deaths'] / data['population']

    return data

In [3]:
data = simulate_death_rate_data(n_cells=50, n_periods=100, true_lambda=0.001, excess_zeros_prob=0.80, random_seed=42)

In [4]:
# Define the Bayesian Hierarchical Zero-Inflated Log-Normal Model
with pm.Model() as model:
    # Priors for the overall mean death rate and its variance
    mu = pm.Normal('mu', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # Hierarchical structure: each cell has its own rate drawn from a common distribution
    cell_mu = pm.Normal('cell_mu', mu=mu, sigma=sigma, shape=len(data['cell_id'].unique()))
    
    # Zero-inflation probability
    psi = pm.Beta('psi', alpha=1, beta=1, shape=len(data['cell_id'].unique()))
    
    # Death rate for each observation
    log_death_rate = cell_mu[data['cell_id'].values]
    
    # Likelihood for non-zero death rates
    non_zero = pm.LogNormal.dist(mu=log_death_rate, sigma=sigma)
    
    # Approximate zero death rates using a very small variance normal distribution centered at zero
    zero = pm.Normal.dist(mu=0, sigma=1e-6)
    
    # Mixture model for zero-inflation
    w = pm.math.stack([psi[data['cell_id'].values], 1 - psi[data['cell_id'].values]], axis=1)
    components = [zero, non_zero]
    likelihood = pm.Mixture('death_rate', w=w, comp_dists=components, observed=data['death_rate'].values)
    
    # Inference
    trace = pm.sample(2000, tune=1000, target_accept=0.95)

# Extract the estimated death rates
posterior_means = trace.posterior['cell_mu'].mean(dim=['chain', 'draw'])
expected_death_rates = pd.DataFrame({
    'cell_id': data['cell_id'].unique(),
    'expected_death_rate': np.exp(posterior_means)
})

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma, cell_mu, psi]


/home/simon/anaconda3/envs/pymc_env/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 30 seconds.


In [5]:
expected_death_rates.head()

,cell_id,expected_death_rate
0,0,0.000986
1,1,0.000800
2,2,0.000763
3,3,0.000853
4,4,0.001071


In [6]:
# Function to calculate return periods
def calculate_return_periods(expected_death_rates, threshold):
    """
    Calculate the return periods for each cell based on the expected death rates.

    Parameters:
    expected_death_rates (pd.DataFrame): DataFrame containing the expected death rates for each cell.
    threshold (float): The threshold death rate for calculating return periods.

    Returns:
    pd.DataFrame: DataFrame containing the return periods for each cell.
    """
    expected_death_rates['prob_exceedance'] = 1 - np.exp(-expected_death_rates['expected_death_rate'] / threshold)
    expected_death_rates['return_period'] = 1 / expected_death_rates['prob_exceedance']
    return expected_death_rates

# Example usage
threshold_death_rate = 0.1
return_periods = calculate_return_periods(expected_death_rates, threshold_death_rate)
return_periods.head()

,cell_id,expected_death_rate,prob_exceedance,return_period
0,0,0.000986,0.009814,101.896100
1,1,0.000800,0.007968,125.504301
2,2,0.000763,0.007600,131.572850
3,3,0.000853,0.008493,117.746552
4,4,0.001071,0.010657,93.836258


In [7]:
# Function to calculate threshold death rate for a given return period
def calculate_threshold_for_return_period(expected_death_rates, return_period):
    """
    Calculate the threshold death rate for each cell given a return period.

    Parameters:
    expected_death_rates (pd.DataFrame): DataFrame containing the expected death rates for each cell.
    return_period (float): The return period for which to calculate the threshold death rate.

    Returns:
    pd.DataFrame: DataFrame containing the threshold death rate for each cell.
    """
    expected_death_rates['threshold_death_rate'] = -expected_death_rates['expected_death_rate'] / np.log(1 - 1/return_period)
    return expected_death_rates

# Example usage
return_period = 100
thresholds = calculate_threshold_for_return_period(expected_death_rates, return_period)
thresholds.head()

,cell_id,expected_death_rate,prob_exceedance,return_period,threshold_death_rate
0,0,0.000986,0.009814,101.896100,0.098130
1,1,0.000800,0.007968,125.504301,0.079597
2,2,0.000763,0.007600,131.572850,0.075912
3,3,0.000853,0.008493,117.746552,0.084864
4,4,0.001071,0.010657,93.836258,0.106604


In [8]:
# Define the Bayesian Hierarchical Model
with pm.Model() as model:
    # Priors for the overall mean death rate and its variance
    mu = pm.Normal('mu', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # Hierarchical structure: each cell has its own rate drawn from a common distribution
    cell_mu = pm.Normal('cell_mu', mu=mu, sigma=sigma, shape=len(data['cell_id'].unique()))
    
    # Likelihood: observed death counts are Poisson-distributed
    death_rate = pm.Deterministic('death_rate', cell_mu[data['cell_id'].values])
    deaths = pm.Poisson('deaths', mu=death_rate * data['population'], observed=data['deaths'])
    
    # Inference
    trace = pm.sample(2000, tune=1000, target_accept=0.95)

# Extract the estimated death rates
posterior_means = trace['cell_mu'].mean(axis=0)
expected_death_rates = pd.DataFrame({
    'cell_id': data['cell_id'].unique(),
    'expected_death_rate': posterior_means
})

print(expected_death_rates.head())

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'mu': array(-0.05840645), 'sigma_log__': array(-0.83487923), 'cell_mu': array([ 0.53494841, -0.27800701,  0.15320151, -0.14510694, -0.13278081,
        0.86309645,  0.03855071,  0.57390374, -0.06101582,  0.74578116,
       -0.28203954,  0.45675429,  0.79586304, -0.32311436, -0.64872517,
       -0.67075243, -0.17362452,  0.33284545, -0.94445559,  0.06631547,
        0.79146579, -0.91172288,  0.09494999,  0.53533403,  0.24427465,
       -0.59709404,  0.73771091,  0.58798035,  0.48170487,  0.10299716,
       -0.05230049, -0.92753861, -0.17408136,  0.91343868,  0.44515585,
       -0.94994671, -0.12915101, -0.32852098,  0.53803908, -0.84060246,
       -0.23962982,  0.24658793, -0.51650963,  0.0288536 ,  0.89335972,
       -0.61421971,  0.23501255, -0.64395328,  0.16209844,  0.48172782])}

Logp initial evaluation results:
{'mu': -3.22, 'sigma': -1.15, 'cell_mu': -43.72, 'deaths': -inf}
You can call `model.

In [ ]:
# Step 1: Data Simulation
np.random.seed(42)

# Simulate data for 1000 grid cells over 100 time periods
n_cells = 50
n_periods = 100
total_observations = n_cells * n_periods

population = np.random.randint(1000, 10000, size=n_cells)
population = np.repeat(population, n_periods)  # Repeat population for each time period

true_lambda = 0.1  # true mean death rate per person
excess_zeros_prob = 0.3  # probability of excess zeros

# Simulate death counts with excess zeros for each cell over time periods
death_counts = np.random.negative_binomial(true_lambda * population, 0.5)
excess_zeros = np.random.binomial(1, excess_zeros_prob, size=total_observations)
death_counts = death_counts * (1 - excess_zeros)

# Create a DataFrame
data = pd.DataFrame({
    'cell_id': np.tile(np.arange(n_cells), n_periods),
    'time_period': np.repeat(np.arange(n_periods), n_cells),
    'population': population,
    'deaths': death_counts
})

# Calculate death rates
data['death_rate'] = data['deaths'] / data['population']

data.sample(10)

In [ ]:
# Step 2: Bayesian Model with PyMC
with pm.Model() as zinb_model:
    
    # Hierarchical priors for the count model parameters
    alpha_mu = pm.Normal('alpha_mu', mu=0, sigma=10)
    alpha_sigma = pm.HalfNormal('alpha_sigma', sigma=10)
    alpha = pm.Normal('alpha', mu=alpha_mu, sigma=alpha_sigma, shape=n_cells)
    
    beta_mu = pm.Normal('beta_mu', mu=0, sigma=10)
    beta_sigma = pm.HalfNormal('beta_sigma', sigma=10)
    beta = pm.Normal('beta', mu=beta_mu, sigma=beta_sigma, shape=n_cells)
    
    # Hierarchical priors for the zero-inflation model parameters
    gamma_mu = pm.Normal('gamma_mu', mu=0, sigma=10)
    gamma_sigma = pm.HalfNormal('gamma_sigma', sigma=10)
    gamma = pm.Normal('gamma', mu=gamma_mu, sigma=gamma_sigma, shape=n_cells)
    
    delta_mu = pm.Normal('delta_mu', mu=0, sigma=10)
    delta_sigma = pm.HalfNormal('delta_sigma', sigma=10)
    delta = pm.Normal('delta', mu=delta_mu, sigma=delta_sigma, shape=n_cells)

    # Linear predictors
    cell_idx = data['cell_id'].values
    log_population = np.log(data['population'])
    mu = pm.math.exp(alpha[cell_idx] + beta[cell_idx] * log_population)
    psi = pm.math.sigmoid(gamma[cell_idx] + delta[cell_idx] * log_population)

    # Likelihood
    deaths = pm.ZeroInflatedNegativeBinomial('deaths', psi=psi, mu=mu, alpha=1.0, observed=data['deaths'])

    # Sample from the posterior
    idata = pm.sample(2000, tune=1000, target_accept=0.95)

In [ ]:
# Step 3: Posterior Analysis
# Summary of the posterior
az.summary(idata, var_names=['alpha', 'beta', 'gamma', 'delta'])

# Extract posterior samples
posterior_alpha = idata.posterior['alpha'].values
posterior_beta = idata.posterior['beta'].values
posterior_gamma = idata.posterior['gamma'].values
posterior_delta = idata.posterior['delta'].values

# Calculate expected death rate for each cell using the posterior samples
log_population = np.array(log_population).reshape(-1, 1)
posterior_mu = np.exp(posterior_alpha[:, None] + posterior_beta[:, None] * log_population)
posterior_psi = 1 / (1 + np.exp(-(posterior_gamma[:, None] + posterior_delta[:, None] * log_population)))


# Expected death rates
expected_deaths = posterior_mu * (1 - posterior_psi)
expected_deaths = expected_deaths.mean(axis=0).mean(axis =1)
expected_death_rate = expected_deaths / population

# Step 4: Calculate Return Periods
# For simplicity, let's assume a threshold death rate for return period calculation
threshold_death_rate = 0.001

# Calculate the probability of exceeding the threshold for each cell
#prob_exceed_threshold = (expected_death_rate > threshold_death_rate).astype(int)

# Calculate the probability of exceeding the observed death rate
observed_death_rate = data['death_rate'].values
prob_exceed_observed = (expected_death_rate > observed_death_rate.reshape(1, n_cells)).mean(axis=0)


# Debug print statements
print("Probability of exceeding threshold:")
print(prob_exceed_threshold)

# Calculate return periods
return_period = 1 / prob_exceed_threshold
return_period[prob_exceed_threshold == 0] = np.inf  # Handle division by zero

# Display results
results = pd.DataFrame({
    'population': data['population'],
    'deaths': data['deaths'],
    'death_rate': data['death_rate'],
    'expected_death_rate': expected_death_rate,
    'return_period': return_period
})

print(results.head(10))


In [ ]:
# Step 1: Data Simulation
np.random.seed(42)

# Simulate data for 1000 grid cells
n_cells = 1000
population = np.random.randint(1000, 10000, size=n_cells)
true_lambda = 0.1  # true mean death rate per person
excess_zeros_prob = 0.3  # probability of excess zeros

# Simulate death counts with excess zeros
death_counts = np.random.negative_binomial(true_lambda * population, 0.5)
excess_zeros = np.random.binomial(1, excess_zeros_prob, size=n_cells)
death_counts = death_counts * (1 - excess_zeros)

# Create a DataFrame
data = pd.DataFrame({
    'population': population,
    'deaths': death_counts
})

# Calculate death rates
data['death_rate'] = data['deaths'] / data['population']

In [ ]:
# Step 2: Bayesian Model with PyMC
with pm.Model() as zinb_model:
    # Priors for the count model parameters
    alpha = pm.HalfNormal('alpha', sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10)
    
    # Priors for the zero-inflation model parameters
    gamma = pm.Normal('gamma', mu=0, sigma=10)
    delta = pm.Normal('delta', mu=0, sigma=10)

    # Linear predictors
    log_population = np.log(data['population'])
    mu = pm.math.exp(alpha + beta * log_population)
    psi = pm.math.sigmoid(gamma + delta * log_population)

    # Likelihood
    deaths = pm.ZeroInflatedNegativeBinomial('deaths', psi=psi, mu=mu, alpha=1.0, observed=data['deaths'])

    # Sample from the posterior
    idata = pm.sample(2000, tune=1000, target_accept=0.95)

In [ ]:
# Step 3: Posterior Analysis
# Summary of the posterior
az.summary(idata, var_names=['alpha', 'beta', 'gamma', 'delta'])

# Extract posterior samples
posterior_alpha = idata.posterior['alpha'].values
posterior_beta = idata.posterior['beta'].values
posterior_gamma = idata.posterior['gamma'].values
posterior_delta = idata.posterior['delta'].values

# Calculate expected death rate for each cell using the posterior samples
log_population = np.array(log_population).reshape(-1, 1)
posterior_mu = np.exp(posterior_alpha[:, None] + posterior_beta[:, None] * log_population)
posterior_psi = 1 / (1 + np.exp(-(posterior_gamma[:, None] + posterior_delta[:, None] * log_population)))


# Expected death rates
expected_deaths = posterior_mu * (1 - posterior_psi)
expected_deaths = expected_deaths.mean(axis=0).mean(axis =1)
expected_death_rate = expected_deaths / population

# Step 4: Calculate Return Periods
# For simplicity, let's assume a threshold death rate for return period calculation
threshold_death_rate = 0.001

# Calculate the probability of exceeding the threshold for each cell
#prob_exceed_threshold = (expected_death_rate > threshold_death_rate).astype(int)

# Calculate the probability of exceeding the observed death rate
observed_death_rate = data['death_rate'].values
prob_exceed_observed = (expected_death_rate > observed_death_rate.reshape(1, n_cells)).mean(axis=0)


# Debug print statements
print("Probability of exceeding threshold:")
print(prob_exceed_threshold)

# Calculate return periods
return_period = 1 / prob_exceed_threshold
return_period[prob_exceed_threshold == 0] = np.inf  # Handle division by zero

# Display results
results = pd.DataFrame({
    'population': data['population'],
    'deaths': data['deaths'],
    'death_rate': data['death_rate'],
    'expected_death_rate': expected_death_rate,
    'return_period': return_period
})

print(results.head(10))
